In [61]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.estimators import MaximumLikelihoodEstimator, HillClimbSearch, BicScore
from pgmpy.inference import VariableElimination
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score


In [62]:
spam_df = pd.read_csv("Downloads\spam.csv")

In [63]:
spam_df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [64]:
spam_df['spam'] = spam_df['Category'].apply(lambda x: 1 if x == 'spam' else 0)


In [65]:
spam_df

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1
5568,ham,Will ü b going to esplanade fr home?,0
5569,ham,"Pity, * was in mood for that. So...any other s...",0
5570,ham,The guy did some bitching but I acted like i'd...,0


In [66]:
spam_df['spam'] = spam_df['Category'].apply(lambda x: 1 if x == 'spam' else 0)


In [67]:
x_train, x_test, y_train, y_test = train_test_split(spam_df.Message, spam_df.spam, test_size=0.2, random_state=42)


In [68]:
x_train

1978    Reply to win £100 weekly! Where will the 2006 ...
3989    Hello. Sort of out in town already. That . So ...
3935     How come guoyang go n tell her? Then u told her?
4078    Hey sathya till now we dint meet not even a si...
4086    Orange brings you ringtones from all time Char...
                              ...                        
3772    Hi, wlcome back, did wonder if you got eaten b...
5191                               Sorry, I'll call later
5226        Prabha..i'm soryda..realy..frm heart i'm sory
5390                           Nt joking seriously i told
860               Did he just say somebody is named tampa
Name: Message, Length: 4457, dtype: object

In [69]:
cv = CountVectorizer(binary=True)
x_train_count = cv.fit_transform(x_train.values)
x_test_count = cv.transform(x_test.values)


In [70]:
model = BayesianNetwork([('Message', 'Spam')])


In [71]:
x_train_df = pd.DataFrame(x_train_count.toarray(), columns=cv.get_feature_names_out())


In [72]:
x_train_df['spam'] = y_train.reset_index(drop=True)


In [73]:
for col in x_train_df.columns:
    x_train_df[col] = x_train_df[col].astype('category')

In [74]:
model = BayesianNetwork([('spam', col) for col in x_train_df.columns if col != 'spam'])


In [75]:
model.fit(x_train_df, estimator= MaximumLikelihoodEstimator)


In [76]:
# Prepare test data
x_test_df = pd.DataFrame(x_test_count.toarray(), columns=cv.get_feature_names_out())

In [77]:
x_test_df_missing = x_test_df.copy()
x_test_df_missing['spam'] = np.nan

In [78]:
# Make predictions on the test set
inference = VariableElimination(model)
y_pred = inference.map_query(variables=['spam'], evidence=x_test_df)



ValueError: Can't have the same variables in both `variables` and `evidence`. Found in both: {'spam'}

In [ ]:
y_pred = np.array(y_pred_dict['spam'])

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)
print("\nAccuracy:", accuracy)